In [31]:
import numpy as np
import random
import pandas as pd
import itertools
import math
import time
import concurrent.futures

In [98]:
class Landscape:
    def __init__(self, smoothness, length=2000):

        self.s = smoothness
        self.length = length

        """Generate landscape heights"""
        total_segments = round(self.length/self.s)
        points = [random.choice(range(1, 101))] 
        heights = []
        
        for j in range(total_segments-1):
            a = points[-1]
            b = random.choice(range(1, 101))
            points.append(b) # random points to define landscape
            seg_len = random.choice(range(1, 2*self.s))
            """Fill in locations between two points"""
            step = np.round((b - a)/seg_len, 2)
            for i in range(1, seg_len+1):
                if a + step*i > 100:
                    heights.append(100)
                else:
                    heights.append(a + step*i)
            
        self.heights = heights
        self.length = len(self.heights)

### Model

In [360]:
class Agent():
    def __init__(self, no, h, landscape, sigma=0, limit=2000):
        self.no = no
        self.h = h # Ordered list of step sizes
        self.landscape = landscape
        self.score = 0 # Highest value found
        self.sigma = sigma # Level of noise in signals
        self.limit = limit
        self.scount = 0
        
    def data(self, loc):
        """Get noisy data from location"""
        return max(np.random.normal(self.landscape.heights[loc], 
                                    self.sigma), 
                   0.001)
    
    def search(self, start, own_hist, in_hist):
        if self.scount >= self.limit:
            return own_hist
        """own_hist records agent's own search results"""
        """in_hist records known results by the ingroup"""
        start = start # Starting location on landscape
        loc = start # Current location
        findings = own_hist
        if (in_hist[loc] == 0): 
            """Starting loc hasn't been searched by ingroup members"""
            value = self.data(loc)
            findings[loc], maxi = value, value
            self.scount += 1
        else:
            """Starting loc has been searched by ingroup members"""
            maxi = in_hist[loc] # Current max value found
        
        count = 0 # Number of step sizes tried
        n_total = 0
        
        while ((count < len(self.h)) & (self.scount < self.limit)):
            nxt = (loc + self.h[n_total%3])%self.landscape.length # Next loc to check
            if in_hist[nxt] == 0:
                """Never checked by ingroup"""
                value = self.data(nxt)
                findings[nxt], in_hist[nxt] = value, value
                if maxi < value:
                    """Found value higher than current one"""
                    loc, maxi, count = nxt, value, 0
                    n_total += 1
                else:
                    count += 1
                    n_total += 1
                self.scount += 1
            else:
                """Loc already checked by self or ingroup"""
                value = in_hist[nxt]
                if maxi < value:
                    loc, maxi, count = nxt, value, 0
                    n_total += 1
                else:
                    count += 1
                    n_total += 1
        print(self.scount)
        return findings

In [ ]:
class Team():
    def __init__(self, members, landscape, trust_level=1):
        self.members = members
        self.landscape = landscape
        self.trust_level = trust_level # Size of trusted subgroups
        self.trust = dict()
        for a in self.members:
            self.trust[a] = [a]
        if self.trust_level > 0:
            M = self.members
            random.shuffle(M)
            k = math.ceil(len(self.members)*self.trust_level)
            while len(M) > 0:
                subgroup = M[:k]
                M = M[k:]
                for b in subgroup:
                    self.trust[b] = subgroup
    
    def aggregate(self, maps):
        """Aggregate search results from multiple agents"""
        denom = np.sum([np.where(m>0, 1, 0) for m in maps.values()], axis=0)
        num = np.sum([m for m in maps.values()], axis=0)
        return num/(denom + np.where(denom==0, 1,0))
                    
    def tournament(self, start):
        maps = dict() # Cumulative search results by members
        for a in self.members:
            maps[a] = np.array([0]*self.landscape.length)
        on = True
        maxi = 0 # Current max value found
        loc = start # Location where current max value is found

        while on:
            for m in self.members:
                in_hist = np.sum([maps[n] for n in self.trust[m]], axis=0)
                maps[m] = m.search(loc, maps[m], in_hist)
                
            on = False
            new_max, new_loc = np.amax(self.aggregate(maps)), np.argmax(self.aggregate(maps))
            if new_max > maxi:
                on = True # Continue if higher value found in new round
                loc, maxi = new_loc, new_max

        return self.landscape.heights[np.argmax(self.aggregate(maps))]

In [ ]:
poolsize = 12
pool = list(range(1, poolsize+1))
h_each = 3 # Number of step sizes each agent has
per_team = 9 # Number of agents per team
all_perm = list(itertools.permutations(pool, r=h_each)) # All possible step size combo
Smoothness = range(1, 9)
sigma = 8
Trust = [0, 0.33, 0.5, 1]
runs = 1

In [ ]:
LS = [] # List of landscapes to test
Experts = dict()
for s in Smoothness:
    for run in range(runs):
        np.random.seed()
        L = Landscape(s)
        LS.append(L)

In [ ]:
def calc_score(a, L):
    """Calculate agent's avg search score from all starting points of landscape"""
    scores = [] 
    for i in range(L.length):
        np.random.seed()
        results = a.search(i, np.array([0]*L.length), np.array([0]*L.length))
        score = L.heights[np.argmax(results)]
        scores.append(score)
    a.score = np.mean(scores)

In [ ]:
def find_experts(L, t):
    """Returns agents with top individual search scores"""
    agents = [] # all possible heuristic profiles
    for i in range(len(all_perm)):
        a = Agent(i, all_perm[i], L, sigma=0)
        agents.append(a)
    if __name__ == '__main__':
        with concurrent.futures.ThreadPoolExecutor() as executor:
            r = [executor.submit(calc_score, a, L) for a in agents]
    agents.sort(key=lambda x: x.score, reverse=True) # sort agents by expertise
    return Team(agents[:per_team], L, trust_level=t)

In [ ]:
def tournament(team, i):
    np.random.seed()
    return team.tournament(i)

In [ ]:
def run():
    cols = ['smoothness', 'diverse', 'expert', 
            'd_heuristics', 'x_heuristics', 
            'trust', 'sigma', 'poolsize']
    df = pd.DataFrame(columns=cols)

    for L in LS:
        for t in Trust: # trust level
            """Creat expert group"""
            expert = find_experts(L, t)
            for e in expert.members:
                e.sigma = sigma
            """Creat diverse group"""
            d_heu = random.sample(all_perm, per_team) # Randomly generate diverse group
            diverse = Team([Agent(i+len(all_perm), d_heu[i], L, sigma) for i in range(per_team)], 
                                            L, 
                                            trust_level=t)
            d_record = []
            x_record = []

            if __name__ == '__main__':
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    results = [executor.submit(tournament, diverse, i) for i in range(L.length)]

                    for f in concurrent.futures.as_completed(results):
                        d_record.append(f.result())

                with concurrent.futures.ThreadPoolExecutor() as executor:
                    results = [executor.submit(tournament, expert, i) for i in range(L.length)]

                    for f in concurrent.futures.as_completed(results):
                        x_record.append(f.result())
           
            df = pd.concat([df, pd.DataFrame([[L.s,
                                              np.mean(d_record), 
                                              np.mean(x_record), 
                                              list(itertools.chain.from_iterable([a.h for a in diverse.members])), 
                                              list(itertools.chain.from_iterable([a.h for a in expert.members])), 
                                              t,
                                              sigma,
                                              poolsize
                                             ]], columns=cols)], ignore_index=True)
    return(df)

### Correlation between function and trust

In [64]:
def cov_sim(set1, set2):
    """count how many step sizes overlap between agents"""
    count = 0
    for i in set1:
        if i in set2:
            count += 1
    return count

In [ ]:
class CorrTeam(Team):
    def __init__(self, members, landscape, trust_level=0.5):
        super().__init__(members, landscape, trust_level)
        self.trust = dict()
        for a in self.members:
            self.trust[a] = [a]
        if self.trust_level > 0:
            M = self.members
            k = math.ceil(len(self.members)*self.trust_level)
            all_h = list(itertools.chain.from_iterable([m.h for m in self.members]))
            while len(M) > 0:
                rmd = sorted(M, key=lambda x: cov_sim(all_h, x.h), reverse=True)[0]
                M.sort(key=lambda x: cov_sim(rmd.h, x.h), reverse=True)
                subgroup = M[:k]
                for b in subgroup:
                    self.trust[b] = subgroup
                M = M[k:]

### Test code

In [398]:
L = Landscape(4)
a1 = Agent(1, random.sample(range(1, 13), 3), L)
a2 = Agent(2, random.sample(range(1, 13), 3), L)
a3 = Agent(3, random.sample(range(1, 13), 3), L)
a4 = Agent(4, random.sample(range(1, 13), 3), L)
a5 = Agent(5, random.sample(range(1, 13), 3), L)
a6 = Agent(6, random.sample(range(1, 13), 3), L)
a7 = Agent(7, random.sample(range(1, 13), 3), L)
a8 = Agent(8, random.sample(range(1, 13), 3), L)
a9 = Agent(9, random.sample(range(1, 13), 3), L)
T = Team([a1, a2, a3, a4, a5, a6, a7, a8, a9], L)